If you are running this on Google Colab, you need to uncomment (remove the `#`) and execute the following lines to install the Pyomo package, the solver, and some helper tools. If you are running this on Binder or elsewhere (e.g. your own computer) you can ignore this.

In [1]:
# !pip install pyomo==6.4.1
# !apt install glpk-utils
# !pip install "git+https://github.com/sjpfenninger/sen1511.git#egg=sen1511utils&subdirectory=sen1511utils"

In [2]:
import pyomo.environ as pyo
from sen1511utils import summarise_results

# Assignment 2 - Linear Programming with Pyomo

This assignment is structured in two blocks: individual tasks and group tasks. The individual tasks are made of simple preliminary exercises that should be done individually before the instruction session. These are necessary/helpful for completing the group tasks. The group tasks involve the use of Python/Pyomo and consist of a complete optimization problem, which will be solved during the instructions.


## Individual tasks

Review the [Getting started with Pyomo](2.a%20-%20Getting%20started%20with%20Pyomo.ipynb) notebook.

## Group tasks

The group tasks involve the use of Python/Pyomo and consist of a complete optimization problem, which will be solved during the instructions.

### Exercise 1

Consider two generating units supplying at least the system demand P_total,D=650 MW for the coming hour (extra power above the system demand will be used for storage, which is assumed to be unlimited). The unit cost function is characterized by the parameters provided in the table below:

| Unit | Generation costs (EUR/MWh) | P_min,G (MW) | P_max,G (MW) |
|:---|---:|---:|---:|
| 1 | 35 | 0 | 600 |
| 2 | 55 | 100 | 300 |


(a) Formulate the above-mentioned economic dispatch problem as linear programming by specifying:<br>
<br>
&nbsp;&nbsp;&nbsp;&nbsp;(i) all decision variables<br>
<br>
&nbsp;&nbsp;&nbsp;&nbsp;(ii) the objective function<br>
<br>
&nbsp;&nbsp;&nbsp;&nbsp;(iii) relevant constraints


<div style="color:red">
<br>
&nbsp;&nbsp;&nbsp;&nbsp;(i) all decision variables<br>  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; PG1, PG2 - Power generated by Unit_1 resp. Unit_2  <br>

&nbsp;&nbsp;&nbsp;&nbsp;(ii) the objective function<br>  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; minimise f(PG1,PG2) = 35 * PG2 <br>  

&nbsp;&nbsp;&nbsp;&nbsp;(iii) relevant constraints <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Demand (+storage) constraint: PG1 + PG2  ≥ 650 <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Capacity constraints: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  0 ≤ PG1 ≤ 600 <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100 ≤ PG2 ≤ 300 <br>
</div>

(b) Solve the described problem graphically (without Python); include in your graph contour lines (isolines) of the objective function. Specify the optimal values for the variables and the objective function.

<div style="color:red">

_See graph_

Iso-lines (contour lines) have the form: PG2 = (-35/55) * PG1 + constant, e.g.: PG2 = (-35/55) * PG1 + 300. Here, the value for the iso-line is chosen as J = 16500; 16500 = 35 * PG1 + 55 * PG2. 

PG2 = (-35/55) * PG1 + 600. Here, the value for the iso-line is chosen as J = 33000; 33000 = 35 * PG1 + 55 * PG2, etc.

The optimum (if it exists) lies on a corner of a feasible area. In our case, at the corners of the feasible area, the objective function values are: f(550, 100) = 24750, f(350, 300) = 28750, f(600, 300) = 37500, f(600, 100) = 2650. 

The minimum is in the point PG1 = 550 and PG2 = 100. Unit_1 should produce 550 MW and Unit_2 should produce 100 MW. The total cost is 24750 [€/h].

</div>

(c) What can you say about the sensitivity of the presented problem? Answer this question without using Python by calculating shadow prices.

<div style="color:red"> 

 
 

**Sensitivity w.r.t. allowable increase/decrease of the (cost) coefficients in the objective function**   

The demand constraint and the capacity constraint for PG2 with lower bound (PG2 ≥ 100) are the active (binding) constraints. To say something about the sensitivity of the minimum with respect to the (cost) coefficients in the objective function, we compare the position of the iso-lines with the position of the demand constraint. We see that if the iso-lines would be parallel to the demand constraint we get the infinite number of solutions (all points on the demand constraint). So, we can conclude that if the cost coefficient of Unit_1 remains below 55, we will get still the same optimum (allowable increase of the cost coefficient for Unit_1 is 20). If the cost coefficient for Unit_1 becomes more than 55, the minimum would change to the point (350, 300). By analogy, we can say that the allowable decreasing of the cost coefficient for Unit_2 is 20 (it should be more than 35) to get the same optimum.   

 
 

The active capacity constraint (PG2 ≥ 100) can be replaced by a lower bound not less than 50, and the optimum will still be at the intersection with the lower-bound capacity constraint for PG2. If the lower bound for PG2 is less than 50, then the upper bound on PG1 would be an active constraint.   

 
 

**Sensitivity with respect to small change in the active constraint**   

 It is often interesting to see to what extent the objective function will change if there is a small change in an active constraint. In Linear Programming (LP), we look for the so-called shadow prices; they tell us how much the objective function will change in case the active constraint is changed by +1 unit. 

 
 

In this case, there are two active constraints (i.e., constraints on which the optimum lies):   

- PG1 + PG2 = 650   

- PG2 = 100   

 
 

To calculate the shadow price for the 1st active constraint, we solve the following optimization problem. 

 
 

</div> 

 

(d) Solve the problem in Python and specifically examine the shadow prices delivered by Pyomo for the optimal solution. Compare your solutions here with your answers above.

In [3]:
model = pyo.ConcreteModel(name = "Primal Problem")
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

model.Unit1 = pyo.Var(domain=pyo.NonNegativeReals)
model.Unit2 = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

model.profit = pyo.Objective(
    expr = 35 * model.Unit1 + 55 * model.Unit2,
    sense = pyo.minimize,
)

##
# 3. Constraints
##

model.demand = pyo.Constraint(expr = model.Unit1 + model.Unit2 >= 650)

model.UpperLimit1 = pyo.Constraint(expr = model.Unit1 <= 600)
model.LowerLimit1 = pyo.Constraint(expr = model.Unit1 >= 0)

model.UpperLimit2 = pyo.Constraint(expr = model.Unit2 <= 300)
model.LowerLimit2 = pyo.Constraint(expr = model.Unit2 >= 100)

# Solve the problem
solver = pyo.SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 24750.0, 'Upper bound': 24750.0, 'Number of objectives': 1, 'Number of constraints': 6, 'Number of variables': 3, 'Number of nonzeros': 7, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.014948844909667969}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [4]:
summarise_results(model)

(e) Formulate and solve the dual problem in Python to calculate the shadow prices yourself. What do you observe regarding the solution of the primal and the dual problem?

<div class="alert alert-block alert-info">

💡 We will call the dual model `dual_model` instead of `model`. This way, the original (primal) model is still available as well if we want to analyse it further.

</div>

In [5]:
dual_model = pyo.ConcreteModel(name = "Dual Problem")
dual_model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

dual_model.z1 = pyo.Var(domain=pyo.NonNegativeReals)
dual_model.z2 = pyo.Var(domain=pyo.NonNegativeReals)
dual_model.z3 = pyo.Var(domain=pyo.NonNegativeReals)
dual_model.z4 = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

dual_model.profit = pyo.Objective(
    expr = 650*dual_model.z1 - 600*dual_model.z2 + 100*dual_model.z3 - 300*dual_model.z4,
    sense = pyo.maximize
)

##
# 3. Constraints
##

dual_model.c1 = pyo.Constraint(expr = dual_model.z1 - dual_model.z2 <= 35)
dual_model.c2 = pyo.Constraint(expr = dual_model.z1 + dual_model.z3 - dual_model.z4 <= 55)

# Solve the problem
solver = pyo.SolverFactory('glpk')
solver.solve(dual_model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 24750.0, 'Upper bound': 24750.0, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 5, 'Number of nonzeros': 6, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.008694171905517578}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [6]:
summarise_results(dual_model)

## Exercise 2

The  U.S.  Environmental  Protection  Agency  has  proposed  its  first-ever  limit  on  carbon  dioxide emissions from new power plants. The EPA’s new regulations will limit carbon dioxide  emissions from future fossil fuel power plants to 400 kg of CO2 per megawatt hour. \
Power Company PC has  in three different control areas generating plants of type A and generating plants  of  type  B,  and  is  finishing  construction  of  two  new  efficient  gas-fired  power  plants,  one  in Control Area 1 and one in Control Area 2 (see Table below with the generating costs and emissions). They will be operational before winter 2016. 


The power company cannot emit more than 1.400.000kg CO2 per hour without high penalties. This limit will not change after putting new power plants into operation. 
The expectation of Coalition for Clean Energy is that the new CO2emission limits will drive up energy prices.  To  what  extend  do  you  agree  with  this  statement  on  the basis  of  the  situation  of  the  power company PC? To answer this question investigate an economic dispatch of the company by taking into account only the old generation plants being already in operation. 
The power company is supplying expected demand of 1500 MW next hour for the three control area together. No plant should be operating at less than 10% of its maximal capacity.

| Generating costs [$/MWh] 	|                                                           	| Control Area<br>CA1 	| Control Area<br>CA2 	| Control Area<br>CA3 	| Max Capacity <br> [MW]              	| CO2 pollution<br>[kg/MWh] 	|
|--------------------------	|-----------------------------------------------------------	|---------------------	|---------------------	|---------------------	|--------------------------------	|---------------------------	|
|                          	| Plant type A <br>(coal-fired) <br>OLD                     	| 30                  	| 32                  	| 35                  	| in CA1,CA2: 500<br>in CA3: 400 	| 1200                      	|
|                          	| Plant type B<br>(oil-based) <br>OLD                       	| 80                  	| 75                  	| 72                  	| in CA1,CA2: 40<br>in CA3: 500  	| 800                       	|
|                          	| Plant type C<br>(gas-fired; <br>underconstruction)<br>NEW 	| 90                  	| 85                  	| -                   	| in CA1: 700<br>in CA2: 600     	| 400                       	|

**(a)** First, we consider the situation with only the two existing plant types A and B. How should the dispatch of these two plants type A and B be done at minimal cost? What are then the production costs? To answer both questions, formulate a linear program, i.e: <br>
<br>
&nbsp;&nbsp;&nbsp;&nbsp;(i) Specify all decision variables<br>
<br>
&nbsp;&nbsp;&nbsp;&nbsp;(ii) Formulate constraints<br>
<br>
&nbsp;&nbsp;&nbsp;&nbsp;(iii) Define the objective function


<div class="alert alert-block alert-info">

💡 Here we are making yet another change to what variable name we give to the model. We simply use `m` which, thanks to its brevity, makes the rest of the model formulation easier to read (and faster to type).

</div>

In [7]:
m = pyo.ConcreteModel(name = "Model 5.a)")
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

m.A1 = pyo.Var(domain=pyo.NonNegativeReals)
m.A2 = pyo.Var(domain=pyo.NonNegativeReals)
m.A3 = pyo.Var(domain=pyo.NonNegativeReals)
m.B1 = pyo.Var(domain=pyo.NonNegativeReals)
m.B2 = pyo.Var(domain=pyo.NonNegativeReals)
m.B3 = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

m.profit = pyo.Objective(
    expr = 30 * m.A1 + 32 * m.A2 + 35 * m.A3 + 80 * m.B1 + 75 * m.B2 + 72 * m.B3,
    sense = pyo.minimize,
)

##
# 3. Constraints
##

m.A1_lower = pyo.Constraint(expr = m.A1 >= 50)
m.A1_upper = pyo.Constraint(expr = m.A1 <= 500)
m.A2_lower = pyo.Constraint(expr = m.A2 >= 50)
m.A2_upper = pyo.Constraint(expr = m.A2 <= 500)
m.A3_lower = pyo.Constraint(expr = m.A3 >= 40)
m.A3_upper = pyo.Constraint(expr = m.A3 <= 400)
m.B1_lower = pyo.Constraint(expr = m.B1 >= 40)
m.B1_upper = pyo.Constraint(expr = m.B1 <= 400)
m.B2_lower = pyo.Constraint(expr = m.B2 >= 40)
m.B2_upper = pyo.Constraint(expr = m.B2 <= 400)
m.B3_lower = pyo.Constraint(expr = m.B3 >= 50)
m.B3_upper = pyo.Constraint(expr = m.B3 <= 500)
m.demand = pyo.Constraint(expr = m.A1 + m.A2 + m.A3 + m.B1 + m.B2 + m.B3 == 1500)
m.emission_limit = pyo.Constraint(expr = 1200 * m.A1 + 1200 * m.A2 + 1200 * m.A3 + 800 * m.B1 + 800 * m.B2 + 800 * m.B3 <= 1400000)

# Solve the problem
solver = pyo.SolverFactory('glpk')
solver.solve(m)

{'Problem': [{'Name': 'unknown', 'Lower bound': 89300.0000000001, 'Upper bound': 89300.0000000001, 'Number of objectives': 1, 'Number of constraints': 15, 'Number of variables': 7, 'Number of nonzeros': 25, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.00944209098815918}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [8]:
summarise_results(m)

<div class="alert alert-block alert-info">

💡 To avoid having to manually specify all the upper and lower limits for the variables as separate constraints, you can also set the *variable bounds* during the creation of the variable as follows:

```python
m.A1 = pyo.Var(domain=pyo.NonNegativeReals, bounds=(50, 500))
m.A2 = pyo.Var(domain=pyo.NonNegativeReals, bounds=(50, 500))
m.A3 = pyo.Var(domain=pyo.NonNegativeReals, bounds=(40, 400))
m.B1 = pyo.Var(domain=pyo.NonNegativeReals, bounds=(40, 400))
m.B2 = pyo.Var(domain=pyo.NonNegativeReals, bounds=(40, 400))
m.B3 = pyo.Var(domain=pyo.NonNegativeReals, bounds=(50, 500))
```

The disadvantage is that you will not get the constraints you replace that way in the list of constrainst when analysing the model with `summarise_lp_results`.
    
</div>

**(b)** Now, we include consideration of the new plant type C. How should the dispatch of all three plants —type A, B and new type C —be done at minimal cost? What are then the production costs? (implement and solve the problem in Python to answer this) 

<div class="alert alert-block alert-info">

💡 We are overwriting the model from 5.a with that of 5.b by also assigning it to the variable `m`.

</div>

In [9]:
m = pyo.ConcreteModel(name = "Model 5.b)")
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

m.A1 = pyo.Var(domain=pyo.NonNegativeReals)
m.A2 = pyo.Var(domain=pyo.NonNegativeReals)
m.A3 = pyo.Var(domain=pyo.NonNegativeReals)
m.B1 = pyo.Var(domain=pyo.NonNegativeReals)
m.B2 = pyo.Var(domain=pyo.NonNegativeReals)
m.B3 = pyo.Var(domain=pyo.NonNegativeReals)
m.C1 = pyo.Var(domain=pyo.NonNegativeReals)
m.C2 = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

m.profit = pyo.Objective(
    expr = 30 * m.A1 + 32 * m.A2 + 35 * m.A3 + 80 * m.B1 + 75 * m.B2 + 72 * m.B3 + 90 * m.C1 + 85 * m.C2,
    sense = pyo.minimize,
)

##
# 3. Constraints
##

m.A1_lower = pyo.Constraint(expr = m.A1 >= 50)
m.A1_upper = pyo.Constraint(expr = m.A1 <= 500)
m.A2_lower = pyo.Constraint(expr = m.A2 >= 50)
m.A2_upper = pyo.Constraint(expr = m.A2 <= 500)
m.A3_lower = pyo.Constraint(expr = m.A3 >= 40)
m.A3_upper = pyo.Constraint(expr = m.A3 <= 400)
m.B1_lower = pyo.Constraint(expr = m.B1 >= 40)
m.B1_upper = pyo.Constraint(expr = m.B1 <= 400)
m.B2_lower = pyo.Constraint(expr = m.B2 >= 40)
m.B2_upper = pyo.Constraint(expr = m.B2 <= 400)
m.B3_lower = pyo.Constraint(expr = m.B3 >= 50)
m.B3_upper = pyo.Constraint(expr = m.B3 <= 500)
m.C1_lower = pyo.Constraint(expr = m.C1 >= 70)
m.C1_upper = pyo.Constraint(expr = m.C1 <= 700)
m.C2_lower = pyo.Constraint(expr = m.C2 >= 60)
m.C2_upper = pyo.Constraint(expr = m.C2 <= 600)
m.demand = pyo.Constraint(expr = m.A1 + m.A2 + m.A3 + m.B1 + m.B2 + m.B3 + m.C1 + m.C2 == 1500)
m.emission_limit = pyo.Constraint(
    expr = 1200 * m.A1 + 1200 * m.A2 + 1200 * m.A3 + 800 * m.B1 + 800 * m.B2 + 800 * m.B3 + 400 * m.C1 + 400 * m.C2 <= 1400000
)

# Solve the problem
solver = pyo.SolverFactory('glpk')
solver.solve(m)

{'Problem': [{'Name': 'unknown', 'Lower bound': 76165.0000000001, 'Upper bound': 76165.0000000001, 'Number of objectives': 1, 'Number of constraints': 19, 'Number of variables': 9, 'Number of nonzeros': 33, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.011208772659301758}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [10]:
summarise_results(m)

,Name,Value
0,profit,76165.000000
,Name,Value
0,A1,500.000000
1,A2,395.000000
2,A3,40.000000
3,B1,40.000000
4,B2,40.000000
5,B3,50.000000
6,C1,70.000000
7,C2,365.000000


**(c)** Specify which constraints are active (binding) in the optimum (5b). What does it mean?

<div style="color:red">
If a constraint is active (also called a binding constraint), then it is satisfied in the optimum as an equality constraint. Equality constraints are always active. The shadow price of an active constraint is different from zero. In total, there are eight active (binding) constraints:  


- Emission constraint   
- Demand constraint  
- Upper bound of A1  
- Lower bound of A3, B1, B2, B3, and C1</div>

**(d)** What are the shadow prices with respect to the demand constraint and emission constraint for (5b)?

<div style="color:red">

The shadow price of 111.5 for the demand constraint is the instantaneous change in the objective value of the optimal solution obtained by relaxing the constraint (changing the right-hand side of the constraint by 1). The shadow price is valid up to the allowable increase or decrease in the constraint.  

The shadow price of -0.006625 for the emission constraint is the change in the objective value of the optimal solution obtained by changing the right-hand side of the constraint by 1, i.e. 1200 * A1 + 1200 * A2 + 1200 * A3 + 800 * B1 + 800 * B2 + 800 * B3 ≤ 1400001. The shadow price is valid up to the allowable increase or decrease in the constraint.  

Interpretation: If the demand will be not 1500 but 1501, the cost function will change by +111.5. If the emission limit will be not 1400000 but 1400001, the cost function will change by -0.0662.
</div>

**(e)** To what extent do you agree with the statement “The expectation is that the new CO2emission limits will drive up energy prices” on the basis of the situation of power company PC?

<div style="color:red">
It is clear that when only production costs will be taken into consideration without consideringfixed costs (investments) and penalties for violating the emission limits, then the case of PCdoes not support the hypothesis that the new CO2 emission limits will drive up energy price.
</div>